In [3]:
import inspect
import json

import asyncio
import aioredis

import nest_asyncio
nest_asyncio.apply()

import rx
from rx.subject import Subject
from rx.core import Observer, Observable, ConnectableObservable, typing
from rx.disposable import Disposable, CompositeDisposable
from rx import operators as ops

from dataclasses import dataclass, field
from typing import Type, Any, List, Tuple, Callable, Optional, Union, Awaitable, Coroutine, NamedTuple

import mape

In [4]:
from mape.redis import PubObserver, SubObservable
from mape.utils import LogObserver
from mape.typing import Message

from dataclasses import dataclass, field

@dataclass
class PayloadTest:
    name: str = 'ciao'
    number: int = 6
    numbers: List = field(default_factory=lambda: [1, 2, 3])

ciao = SubObservable(["A", "B"])
a = ciao.subscribe(LogObserver("A, B"))

ciao = SubObservable("B")
b = ciao.subscribe(LogObserver("B"))

pub_observer_a = PubObserver("A")
pub_observer_a.on_next(Message(value='a', src='src.A'))
pub_observer_a.on_next('a')
# pub_observer.on_error(Message(value="error", src='src2'))
# pub_observer.on_completed()

await asyncio.sleep(4)
# pub_observer.dispose()

print("B")

pub_observer_b = PubObserver("B")
pub_observer_b.on_next(Message(value=PayloadTest(), src='src.B'))
pub_observer_b.on_next(20)
# pub_observer.on_error(Message(value="error", src='src20'))
# pub_observer.on_completed()

await asyncio.sleep(4)
pub_observer_b.on_next('b1')
await asyncio.sleep(0.1)
pub_observer_b.on_next('b2')
await asyncio.sleep(0)
pub_observer_b.on_next('b3')
pub_observer_a.on_next('a1')
await asyncio.sleep(10)
pub_observer_a.on_next('a2')
await asyncio.sleep(1)

a.dispose()
b.dispose()

(9672) on next: Message(a, src.A, None, 0, 16:45:17.929) | A, B
(9672) on next: a | A, B
B
(9672) on next: Message(PayloadTest(name='ciao', number=6, numbers=[1, 2, 3]), src.B, None, 0, 16:45:21.931) | A, B
(8537) on next: Message(PayloadTest(name='ciao', number=6, numbers=[1, 2, 3]), src.B, None, 0, 16:45:21.931) | B
(9672) on next: 20 | A, B
(8537) on next: 20 | B
(8537) on next: b1 | B
(9672) on next: b1 | A, B
(9672) on next: b2 | A, B
(8537) on next: b2 | B
(9672) on next: b3 | A, B
(8537) on next: b3 | B
(9672) on next: a1 | A, B
(9672) on next: a2 | A, B


In [4]:
""" MANAGE TASK EXCEPTION """

import asyncio
import aioredis
import mape

import nest_asyncio
nest_asyncio.apply()

async def test_raise_unmanaged(param=None):
    while True:
        print(param)
        raise Exception('unmanaged')
        await asyncio.sleep(2)

@mape.utils.log_task_exception
async def test_raise_unmanaged_wrapped(param=None):
    while True:
        print(param)
        raise Exception('wrapped')
        await asyncio.sleep(2)

task = asyncio.create_task(test_raise_unmanaged('silent exception'))
await asyncio.sleep(4)
print("cancel")
task.cancel()

task = asyncio.create_task(mape.utils.task_exception(test_raise_unmanaged_wrapped('wrapped')))
await asyncio.sleep(4)
print("cancel")
task.cancel()

task = asyncio.create_task(test_raise_unmanaged_wrapped('decorated'))
await asyncio.sleep(4)
print("cancel")
task.cancel()

await asyncio.sleep(4)

silent exception


wrapped
Traceback (most recent call last):
  File "/home/muttley/Projects/SecondLevelDegrees/courses/Thesis/source/playground/mape/utils.py", line 129, in task_exception
    return await awaitable
  File "/tmp/ipykernel_176147/836816186.py", line 18, in test_raise_unmanaged_wrapped
    raise Exception('wrapped')
Exception: wrapped


cancel
wrapped


wrapped
Traceback (most recent call last):
  File "/home/muttley/Projects/SecondLevelDegrees/courses/Thesis/source/playground/mape/utils.py", line 129, in task_exception
    return await awaitable
  File "/tmp/ipykernel_176147/836816186.py", line 18, in test_raise_unmanaged_wrapped
    raise Exception('wrapped')
Exception: wrapped


cancel
decorated
cancel


In [5]:
!pip install git+https://github.com/plataux/purse.git

  Cloning https://github.com/plataux/purse.git to /tmp/pip-req-build-plpkjuz9
  Running command git clone -q https://github.com/plataux/purse.git /tmp/pip-req-build-plpkjuz9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 13.7 MB 12.2 MB/s eta 0:00:01
  Created wheel for purse: filename=purse-0.21-py3-none-any.whl size=19446 sha256=cab766f68cdbfba7678a89e4d0e84bd3ab0c733820f9f96c6d68b2096eb9675d
  Stored in directory: /tmp/pip-ephem-wheel-cache-lk0moyfo/wheels/b9/d2/8f/8baa5e4e0778b75c9283a9117d544eb81a58843a4a41a72a66
Successfully built purse
You should consider upgrading via the '/home/muttley/.pyenv/versions/3.8.12/envs/venv-3.8.12/bin/python3.8 -m pip install --upgrade pip' command.


In [10]:
import asyncio
import aioredis

import nest_asyncio
nest_asyncio.apply()

In [ ]:
async def main():
    local_list = ['a', 'b', 'c', 'd', 'e', 'f']

    # local Redis >= 6.0.x plain connection with default params
    red_con = aioredis.Redis()
    redis_key = 'redis_list'

    # The value_type defines the class to serialize to and from
    redis_list = RedisList(redis=red_con, rkey=redis_key, value_type=str)

    # Clear the list, in case it was previously populated
    await redis_list.clear()

    # extend a Redis list with a Python list
    await redis_list.extend(local_list)

    # async list comprehension
    print([x async for x in redis_list])

    # contains
    print(await redis_list.contains('f'))  # True
    print(await redis_list.contains('g'))  # False

    # getting the index of a value
    print(await redis_list.index('c'))  # 2
    print(await redis_list.index('g'))  # None, unlike a Python list that raises a ValueError

    # slicing
    print(await redis_list.slice(2, 5))  # ['c', 'd', 'e']

    # inserting values
    await redis_list.insert(2, 'x')
    await redis_list.insert(-2, 'y')

    # getitem
    assert await redis_list.getitem(2) == 'x'
    assert await redis_list.getitem(-3) == 'y'

    # some deque methods
    await redis_list.appendleft('z')
    await redis_list.pop()
    await redis_list.popleft()

asyncio.run(main())

In [3]:
from pydantic import BaseModel

class Bau:
    def __init__(self, boh) -> None:
        self.ok2 = True
        self.boh = boh

class Ciao:
    def __init__(self) -> None:
        super().__init__()
        self.ok = True
        self.bau = Bau(3)

# Pydantic Model
class Plant(BaseModel):
    name: str = 'undefined'
    healthiness: float
    tasty: bool = False

class Three(Plant):
    color: str = 'green'

In [ ]:
async def main():
    redis = aioredis.Redis()

    k = RedisKeySpace(redis=redis, prefix='k:__app:', value_type=bytes)
    pickled_obj = pickle.dumps(Ciao())
    ret = await k.set('pickled', pickled_obj)
    print(ret)
    ret = await k.get('pickled')
    print(ret)
    unpickled_obj = pickle.loads(ret)
    print(unpickled_obj)

    print("-----------------"*3)

    k = RedisKeySpace(redis=redis, prefix='k:__app:', value_type=Three)
    obj = Three(healthiness = 6.6)
    ret = await k.set('pydantic', obj)
    print(ret)
    ret = await k.get('pydantic')
    print(ret)


asyncio.run(main())

In [12]:
import asyncio
import aioredis
from mape.knowledge import Knowledge

async def main():
    redis = aioredis.Redis()
    k = Knowledge(redis, '__prefix')

    obj = Three(healthiness = 6.6)
    ret = await k.keyspace.set('pickled', obj)
    print(ret)
    ret = await k.keyspace.get('pickled')
    print(ret)

    print("-----------------"*3)

    myset = k.create_set('myset', Three)
    obj = Three(healthiness = 6.6)
    ret = await myset.add(Three(healthiness = 6.6))
    print(ret)
    ret = await myset.len()
    print(ret)
    ret = await myset.update(Three(healthiness = 6.7), Three(healthiness = 6.8), Three(healthiness = 6.8))
    print(ret)
    ret = await myset.len()
    print(ret)
    async for i in myset.values():
        print(i)

asyncio.run(main())

True
name='undefined' healthiness=6.6 tasty=False color='green'
---------------------------------------------------
0
3
0
3
name='undefined' healthiness=6.6 tasty=False color='green'
name='undefined' healthiness=6.7 tasty=False color='green'
name='undefined' healthiness=6.8 tasty=False color='green'


In [13]:
async def main():
    redis = aioredis.Redis()
    k = Knowledge(redis, '__prefix')

    ret = await k.keyspace.get('pickled')
    print(ret)

asyncio.run(main())

name='undefined' healthiness=6.6 tasty=False color='green'
